In [1]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser


embedding_function = AzureOpenAIEmbeddings(model="Text-embedding")

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [2]:
import requests
import os
url = 'https://dev203505.service-now.com/api/now/table/kb_knowledge'

user = 'admin'
pwd = 'FQZxLy^5n9!z'

# Set proper headers
headers = {"Content-Type":"application/json","Accept":"application/json"}

# Do the HTTP request
response = requests.get(url, auth=(user, pwd), headers=headers )

# Check for HTTP codes other than 200
if response.status_code != 200: 
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
    exit()

# Decode the JSON response into a dictionary and use the data
data = response.json()
print(data)

{'result': [{'short_description': 'How to Deal with Spam', 'roles': '', 'wiki': None, 'direct': 'false', 'rating': '', 'description': '', 'generated_with_now_assist': 'false', 'source': '', 'sys_updated_on': '2014-12-19 15:54:36', 'disable_suggesting': 'false', 'sys_class_name': 'kb_knowledge', 'number': 'KB0000011', 'sys_id': '0b48fd75474321009db4b5b08b9a71c2', 'use_count': '0', 'sys_updated_by': 'johnoliver.mendoza', 'flagged': 'false', 'disable_commenting': 'false', 'sys_created_on': '2014-09-09 23:31:10', 'sys_domain': {'link': 'https://dev203505.service-now.com/api/now/table/sys_user_group/global', 'value': 'global'}, 'valid_to': '2100-01-01', 'retired': '', 'workflow_state': 'published', 'text': '<p><span style="font-size: 18pt;"><strong>How to Deal with Spam</strong></span></p>\n<p>Spam\xa0has increasingly become a problem on the Internet. While every Internet user receives some spam, email addresses posted to web sites or in newsgroups and chat rooms attract the most spam.</p>\

In [3]:
data['result'][0]['text']

'<p><span style="font-size: 18pt;"><strong>How to Deal with Spam</strong></span></p>\n<p>Spam\xa0has increasingly become a problem on the Internet. While every Internet user receives some spam, email addresses posted to web sites or in newsgroups and chat rooms attract the most spam.</p>\n<p>To reduce the amount of spam you receive:</p>\n<ul><li>Don&#39;t reply to spam</li><li>Be careful releasing your email address, and know how it will be used</li><li>Be proactive \xa0<br /><br /></li></ul>\n<p style="font-size: 12pt;"><strong>Don&#39;t reply to spam</strong></p>\n<p>If you reply to spam, the spammer or the automated program on the other end will then know that your address is connected to a live person, and the spammer will then bombard you with even more spam, and circulate your address to other spammers. It is critical that you pause and think before replying to any spam. Consider the following guidelines:</p>\n<ul><li>Setting up your email account to generate automatic responses 

In [4]:
from bs4 import BeautifulSoup

def convert_html_to_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    for br in soup.find_all("br"):
        br.replace_with("\n")
    text = soup.get_text(separator="\n")
    text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])
    return text

In [5]:
value = convert_html_to_text(data['result'][0]['text'])

In [6]:
print(value)

How to Deal with Spam
Spam has increasingly become a problem on the Internet. While every Internet user receives some spam, email addresses posted to web sites or in newsgroups and chat rooms attract the most spam.
To reduce the amount of spam you receive:
Don't reply to spam
Be careful releasing your email address, and know how it will be used
Be proactive
Don't reply to spam
If you reply to spam, the spammer or the automated program on the other end will then know that your address is connected to a live person, and the spammer will then bombard you with even more spam, and circulate your address to other spammers. It is critical that you pause and think before replying to any spam. Consider the following guidelines:
Setting up your email account to generate automatic responses while you are away can have the unfortunate side-effect of verifying your email address to every spammer that sends you spam.
If the message appears to come from a legitimate company, the company may have obta

In [7]:
def createDocuments():
    docs=[]
    for kb_article in data['result']:
        metadata = {"sys_id":kb_article['sys_id'], "short_description":kb_article['short_description'], "Number":kb_article["number"]}
        content = convert_html_to_text(kb_article['text'])
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

In [8]:
path = './kb_articles_db'
isdir = os.path.isdir(path)
if(not isdir):
    db2 = Chroma.from_documents(documents=createDocuments(), embedding=embedding_function, persist_directory=path)

In [9]:
# load from disk

query = "I have a issue with outlook? how can i solve it?"

template = """Return only the issue from the user query. And no premable or explaination.

Example:

user query = My outlook is not working. How can I solve it?
response = My outlook is not working.

user query = {query}
"""

prompt_template = PromptTemplate(template=template, input_variables=["query"])
chain = prompt_template | model | StrOutputParser()
response = chain.invoke({"query": query})

print(response)

db3 = Chroma(persist_directory="./kb_articles_db", embedding_function=embedding_function)
docs = db3.similarity_search_with_relevance_scores(response, 4)
print(docs[0])

I have a issue with outlook.
(Document(metadata={'Number': 'KB99999999', 'short_description': 'Microsoft Outlook Issues', 'sys_id': 'c85cd2519f77230088aebde8132e70c2'}, page_content='Microsoft Outlook Issues\nThis article explains how to use automatic replies in Outlook 2010 for Exchange accounts.\nSetting Up Automatic Replies\nClick the\nFile\ntab.\nClick\nAutomatic Replies\n.\nSelect\nSend automatic replies\n.\nIf desired, select the\nOnly send during this time range\ncheck box to schedule when your out of office replies are active. If you do not specify a start and end time, auto-replies will be sent until you select the\nDo not send automatic replies\ncheck box.\nOn the\nInside My Organization\ntab, type the response that you want to send to colleagues while you are out of the office.\nOn the\nOutside My Organization\ntab, select the\nAuto-reply to people outside my organization\ncheck box, and then type the response that you want to send while you are out of the office. Select whe

In [10]:
docs

[(Document(metadata={'Number': 'KB99999999', 'short_description': 'Microsoft Outlook Issues', 'sys_id': 'c85cd2519f77230088aebde8132e70c2'}, page_content='Microsoft Outlook Issues\nThis article explains how to use automatic replies in Outlook 2010 for Exchange accounts.\nSetting Up Automatic Replies\nClick the\nFile\ntab.\nClick\nAutomatic Replies\n.\nSelect\nSend automatic replies\n.\nIf desired, select the\nOnly send during this time range\ncheck box to schedule when your out of office replies are active. If you do not specify a start and end time, auto-replies will be sent until you select the\nDo not send automatic replies\ncheck box.\nOn the\nInside My Organization\ntab, type the response that you want to send to colleagues while you are out of the office.\nOn the\nOutside My Organization\ntab, select the\nAuto-reply to people outside my organization\ncheck box, and then type the response that you want to send while you are out of the office. Select whether you want replies sent t

In [13]:
docs[0][1]

0.21292134104298965